Scroll a specific element?


In [5]:
function scrollClient(client, selector, up = false) {
    // scroll to bottom of messages
    return client
        .execute((selector, up = false) => {
            if (selector === '' || selector === false) {
                window.scroll(window.scrollX, window.scrollY + (up ? -100000 : 100000));
                return;
            }
            var people = document.evaluate(
                selector,
                document, null,
                XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
            people.scrollTop = people.scrollTop + (up ? -100000 : 100000);
        }, selector, up)
};
module.exports = scrollClient;



[Function: scrollClient]

Get all elements until?



In [7]:
var importer = require('../Core');
var {
    getAllXPath, scrollClient
} = importer.import([
    'all elements xpath',
    'scroll specific element'
])

function getAllUntil(client, scrollableSelector,
                     dataSelector,
                     set = [],
                     compare = (a, b) => a === b,
                     cb = (i) => i < 3,
                     up = false,
                     i = 0) {
    return getAllXPath(dataSelector)
        .then(r => {
            var newPosts = ((typeof r === 'string' ? [r] : r) || [])
                .filter(e => set
                    .filter(m => compare(e, m)).length === 0);
            set = newPosts.concat(set);
            return newPosts.length > 0
                ? scrollClient(client, scrollableSelector, up)
                    .pause(4000)
                    .then(() => cb(i)
                        ? getAllUntil(scrollableSelector, dataSelector, set, compare, cb, up, i + 1)
                        : set)
                : Promise.resolve(set);
        })
        .catch(e => console.log(e))
};


module.exports = getAllUntil;




[Function: getAllUntil]

click spa link?


In [ ]:
function clickSpa(client, profile) {
    return client
        .getUrl()
        .then(url => url.indexOf(profile) > -1
              ? []
              : client
             .execute(url => {
                var a = document.createElement('a');
                a.setAttribute('href', url);
                a.setAttribute('class', 'spa');
                a.onclick = () => setTimeout(() => document.body.removeChild(a), 500)
                a.href = url;
                a.style.position = 'absolute';
                a.style.zIndex = 4294967295;
                a.style.top = 0;
                a.style.left = 0;
                a.style.bottom = 0;
                a.style.right = 0;
                document.body.appendChild(a);
            }, profile)
        .click('a.spa[href*="' + profile + '"]')
        .pause(1000))
        .catch(e => console.log(e))
};

module.exports = clickSpa;


Find the element rect dimensions that match this OCR detection of text content on page:

https://www.mathworks.com/help/vision/examples/automatically-detect-and-recognize-text-in-natural-images.html?s_tid=gn_loc_drop

